In [1]:
from datasets import load_dataset
import torch
from pydvl.influence.torch import EkfacInfluence

imdb = load_dataset("imdb")

/Users/fabio/miniconda3/envs/pydvl_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset imdb (/Users/fabio/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 410.90it/s]


In [2]:
small_train_dataset = (
    imdb["train"].shuffle(seed=42).select([i for i in list(range(60))])
)
small_test_dataset = imdb["test"].shuffle(seed=4).select([i for i in list(range(15))])

Loading cached shuffled indices for dataset at /Users/fabio/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-9c48ce5d173413c7.arrow
Loading cached shuffled indices for dataset at /Users/fabio/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-d00218895ddb9236.arrow


In [3]:
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("assemblyai/distilbert-base-uncased-sst2")
model = AutoModelForSequenceClassification.from_pretrained(
    "assemblyai/distilbert-base-uncased-sst2"
)

tokenized_segments = tokenizer(
    ["Pydvl is the best data valuation library, and it is fully open-source!"],
    return_tensors="pt",
    padding=True,
    truncation=True,
)
tokenized_segments_input_ids, tokenized_segments_attention_mask = (
    tokenized_segments.input_ids,
    tokenized_segments.attention_mask,
)
model_predictions = F.softmax(
    model(
        input_ids=tokenized_segments_input_ids,
        attention_mask=tokenized_segments_attention_mask,
    )["logits"],
    dim=1,
)

print("Positive probability: " + str(model_predictions[0][1].item() * 100) + "%")
print("Negative probability: " + str(model_predictions[0][0].item() * 100) + "%")

Positive probability: 99.57591891288757%
Negative probability: 0.42408257722854614%


In [4]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)


tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /Users/fabio/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-844f976fdb8a2b78.arrow
Loading cached processed dataset at /Users/fabio/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-7a61d4de4e064d18.arrow


In [5]:
class ImdbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, attn_mask, labels):
        self.encodings = encodings
        self.attn_mask = attn_mask
        self.labels = labels

    def __getitem__(self, idx):
        x = torch.tensor([self.encodings[idx], self.attn_mask[idx]])
        y = torch.tensor(self.labels[idx])
        return x, y

    def __len__(self):
        return len(self.labels)

In [6]:
train_dataset = ImdbDataset(
    tokenized_train["input_ids"],
    tokenized_train["attention_mask"],
    tokenized_train["label"],
)
test_dataset = ImdbDataset(
    tokenized_test["input_ids"],
    tokenized_test["attention_mask"],
    tokenized_test["label"],
)

In [7]:
class PatchedModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x[:, 0], x[:, 1])["logits"]

In [8]:
for param in model.named_parameters():
    param[1].requires_grad = False

for m_name, module in model.named_modules():
    if len(list(module.children())) == 0 and len(list(module.parameters())) > 0:
        if isinstance(module, torch.nn.Linear):
            for p_name, param in module.named_parameters():
                if (
                    "ffn" in m_name
                    or "pre_classifier" in m_name
                    or "classifier" in m_name
                ):
                    param.requires_grad = True

In [9]:
for param in model.named_parameters():
    if param[1].requires_grad:
        print(param[0], param[1].shape)

distilbert.transformer.layer.0.ffn.lin1.weight torch.Size([3072, 768])
distilbert.transformer.layer.0.ffn.lin1.bias torch.Size([3072])
distilbert.transformer.layer.0.ffn.lin2.weight torch.Size([768, 3072])
distilbert.transformer.layer.0.ffn.lin2.bias torch.Size([768])
distilbert.transformer.layer.1.ffn.lin1.weight torch.Size([3072, 768])
distilbert.transformer.layer.1.ffn.lin1.bias torch.Size([3072])
distilbert.transformer.layer.1.ffn.lin2.weight torch.Size([768, 3072])
distilbert.transformer.layer.1.ffn.lin2.bias torch.Size([768])
distilbert.transformer.layer.2.ffn.lin1.weight torch.Size([3072, 768])
distilbert.transformer.layer.2.ffn.lin1.bias torch.Size([3072])
distilbert.transformer.layer.2.ffn.lin2.weight torch.Size([768, 3072])
distilbert.transformer.layer.2.ffn.lin2.bias torch.Size([768])
distilbert.transformer.layer.3.ffn.lin1.weight torch.Size([3072, 768])
distilbert.transformer.layer.3.ffn.lin1.bias torch.Size([3072])
distilbert.transformer.layer.3.ffn.lin2.weight torch.Size(

In [40]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=5, shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=3, shuffle=True)

In [11]:
patched_model = PatchedModel(model)
patched_model.eval()

ekfac_influence_model = EkfacInfluence(
    patched_model,
    hessian_regularization=0.1,
)
ekfac_influence_model = ekfac_influence_model.fit(train_dataloader)
ekfac_influence_model.update_diag(train_dataloader)

In [41]:
first_test_batch = next(iter(test_dataloader))
first_train_batch = next(iter(train_dataloader))

In [42]:
first_train_batch[1][0] = 1 - first_train_batch[1][0]

In [43]:
ekfac_train_influences = ekfac_influence_model.influences(
    *first_test_batch, *first_train_batch, mode="up"
)

/Users/fabio/miniconda3/envs/pydvl_env/lib/python3.9/site-packages/transformers/models/distilbert/modeling_distilbert.py:222: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::masked_fill.Tensor. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/functorch/BatchedFallback.cpp:84.)
  scores = scores.masked_fill(


In [44]:
torch.mean(ekfac_train_influences, axis=0)

tensor([-5.8491e+01,  1.5467e+00, -8.5055e-03,  9.9609e-01, -5.1906e+00])

In [16]:
first_train_batch[0][0] = first_test_batch[0][0]
first_train_batch[1][0] = first_test_batch[1][0]

In [17]:
ekfac_train_influences = ekfac_influence_model.influences(
    *first_test_batch, *first_train_batch, mode="up"
)

In [18]:
first_train_batch[1][0]

tensor(1)

In [19]:
ekfac_train_influences[0][0]

tensor(3356.6882)

In [20]:
torch.mean(ekfac_train_influences[0])

tensor(796.5195)

In [21]:
ekfac_train_influences.shape

torch.Size([3, 5])

In [29]:
ekfac_train_influences = ekfac_influence_model.influences_by_layer(
    *first_test_batch, *first_train_batch, mode="up"
)

In [31]:
influences = torch.zeros(size=(3, 5))
for layer_id, value in ekfac_train_influences.items():
    influences += value

In [38]:
torch.mean(influences, axis=0) * (598.4626 / 48.8362)

tensor([ 598.4623,   27.0340,    4.7846, 1015.5076,  -10.5945],
       grad_fn=<MulBackward0>)

In [39]:
(598.4626 / 48.8362)

12.254487449883488